## Imports

In [ ]:
import os
import random
import inspect

import numpy as np
import pandas as pd
from shapflow.flow import GraphExplainer, CausalLinks, build_feature_graph

from autograd import Value, populate_dataframe, build_causal_graph, draw_computational_graph

## Constants

**Message from Arijit:**

Common adsorbent for all: SIV

CH4 & 323K: JRY

CH4 & 373K: NPT

CO2 & 323K: STI

CO2 & 373K: RWR

N2 & 323K: GON

N2 & 373K: MFS

In [ ]:
TEMPERATURE = 373 # Can be either 323 or 373
PRESSURE_IDX = 3 # Setting to 0 will index the minimum pressure value
ADSORBATE = 'N2'
BACKGROUND_ADSORBENT = 'MOR'
FOREGROUND_ADSORBENT = 'IWS'
GENERATE_EVERY_PERMUTATION_OF_ADSORBENTS_FLAG = True

In [ ]:
ZEOLITE_LOC_STR = "data/zeolite/complete_data_avail.xlsx"
_ZEOLITE_DATA = pd.read_excel(ZEOLITE_LOC_STR)
ADSORBATE_LOC_STR = f"data/zeolite/{ADSORBATE}.xlsx"
ADSORBATE_DATA: dict = pd.read_excel(ADSORBATE_LOC_STR, sheet_name=None)
RESULTS_FILE_PATH = f'data/zeolite/{ADSORBATE}_RESULTS_{str(TEMPERATURE)}.npy'

In [ ]:
with open(RESULTS_FILE_PATH, 'rb') as RESULTS_FILE:
    X_train = np.load(RESULTS_FILE, allow_pickle=True)
    X_test = np.load(RESULTS_FILE, allow_pickle=True)
    Y = np.load(RESULTS_FILE, allow_pickle=True)
    Y_pred = np.load(RESULTS_FILE, allow_pickle=True)
    y_train = np.load(RESULTS_FILE, allow_pickle=True)
    y_test = np.load(RESULTS_FILE, allow_pickle=True)
    Y_pred_train = np.load(RESULTS_FILE, allow_pickle=True)
    Y_pred_test = np.load(RESULTS_FILE, allow_pickle=True)
    P_TRAIN_STD = np.load(RESULTS_FILE, allow_pickle=True)

In [ ]:
P_TRAIN_STD

## Equations

TODO List:
1. Confirm that the equations are correct

### CH4 Equations

In [ ]:
def predict_CH4_323K(
        P, density, spg, volume, SiOSi_average, SiOSi_gmean, SiOSi_hmean, SiOSi_max, SiOSi_mean, SiOSi_min,
        SiOSi_skew, SiOSi_std, SiOSi_var, SiO_average, SiO_gmean, SiO_hmean, SiO_max, SiO_mean, SiO_min,
        SiO_skew, SiO_std, SiO_var, ASA, AV, NASA, NAV, VolFrac, largest_free_sphere, largest_included_sphere,
        largest_included_sphere_free, max_dim, min_dim, mode_dim, pld, lcd, void_frac, surf_area_m2g, surf_area_m2cm3
    ):
    Value.reset()

    # input variables
    P = Value(P, label='P')
    density = Value(density, label='density')
    spg = Value(spg, label='spg')
    volume = Value(volume, label='volume')
    SiOSi_average = Value(SiOSi_average, label='SiOSi_average')
    SiOSi_gmean = Value(SiOSi_gmean, label='SiOSi_gmean')
    SiOSi_hmean = Value(SiOSi_hmean, label='SiOSi_hmean')
    SiOSi_max = Value(SiOSi_max, label='SiOSi_max')
    SiOSi_mean = Value(SiOSi_mean, label='SiOSi_mean')
    SiOSi_min = Value(SiOSi_min, label='SiOSi_min')
    SiOSi_skew = Value(SiOSi_skew, label='SiOSi_skew')
    SiOSi_std = Value(SiOSi_std, label='SiOSi_std')
    SiOSi_var = Value(SiOSi_var, label='SiOSi_var')
    SiO_average = Value(SiO_average, label='SiO_average')
    SiO_gmean = Value(SiO_gmean, label='SiO_gmean')
    SiO_hmean = Value(SiO_hmean, label='SiO_hmean')
    SiO_max = Value(SiO_max, label='SiO_max')
    SiO_mean = Value(SiO_mean, label='SiO_mean')
    SiO_min = Value(SiO_min, label='SiO_min')
    SiO_skew = Value(SiO_skew, label='SiO_skew')
    SiO_std = Value(SiO_std, label='SiO_std')
    SiO_var = Value(SiO_var, label='SiO_var')
    ASA = Value(ASA, label='ASA')
    AV = Value(AV, label='AV')
    NASA = Value(NASA, label='NASA')
    NAV = Value(NAV, label='NAV')
    VolFrac = Value(VolFrac, label='VolFrac')
    largest_free_sphere = Value(largest_free_sphere, label='largest_free_sphere')
    largest_included_sphere = Value(largest_included_sphere, label='largest_included_sphere')
    largest_included_sphere_free = Value(largest_included_sphere_free, label='largest_included_sphere_free')
    max_dim = Value(max_dim, label='max_dim')
    min_dim = Value(min_dim, label='min_dim')
    mode_dim = Value(mode_dim, label='mode_dim')
    pld = Value(pld, label='pld')
    lcd = Value(lcd, label='lcd')
    void_frac = Value(void_frac, label='void_frac')
    surf_area_m2g = Value(surf_area_m2g, label='surf_area_m2g')
    surf_area_m2cm3 = Value(surf_area_m2cm3, label='surf_area_m2cm3')

    # constants
    beta1 = Value(-0.4054507842872319, label='beta1')
    beta2 = Value(-3.5824569383000946, label='beta2')
    beta3 = Value(3.9932667095050705, label='beta3')
    beta4 = Value(0.5222507180877926, label='beta4')
    beta5 = Value(0.004795464592376811, label='beta5')
    beta6 = Value(-0.1129628499436825, label='beta6')
    beta7 = Value(0.013835556053187287, label='beta7')
    beta8 = Value(-0.09608874134668977, label='beta8')
    intercept = Value(0.0, label='intercept')

    # f terms
    f1 = ((largest_included_sphere*NAV.exp()*(-surf_area_m2cm3).exp()) * P / 
        (1 + (largest_included_sphere*NAV.exp()*(-surf_area_m2cm3).exp()) * P) - 
        0.07469040214406462)/0.23351307544167757
    f1.label = 'f1'

    f2 = ((VolFrac.exp()/(-AV).exp()) * P / 
        (1 + (VolFrac.exp()/(-AV).exp()) * P) - 
        0.593439038924528)/0.19855903035643785
    f2.label = 'f2'

    f3 = ((void_frac.exp()) * P / 
        (1 + (void_frac.exp()) * P) - 
        0.6258918477237851)/0.19909573031440259
    f3.label = 'f3'

    f4 = ((AV/VolFrac.exp()*void_frac) * P / 
        (1 + (AV/VolFrac.exp()*void_frac) * P) - 
        0.026347531768028034)/0.033050115887121194
    f4.label = 'f4'

    f5 = (((-NASA).exp()*(surf_area_m2cm3**2)*spg) * P / 
        (1 + ((-NASA).exp()*(surf_area_m2cm3**2)*spg) * P) - 
        0.451590663829706)/0.4976497103725683
    f5.label = 'f5'

    f6 = (((surf_area_m2cm3**2)*(-volume).exp()*volume) * P / 
        (1 + ((surf_area_m2cm3**2)*(-volume).exp()*volume) * P) - 
        0.04040880811128878)/0.05992104058313283
    f6.label = 'f6'

    f7 = ((1/surf_area_m2g.exp()/pld) * P / 
        (1 + (1/surf_area_m2g.exp()/pld) * P) - 
        0.033471221601783376)/0.11285951143033723
    f7.label = 'f7'

    f8 = ((VolFrac**2/(-pld).exp()) * P / 
        (1 + (VolFrac**2/(-pld).exp()) * P) - 
        0.3805783317820693)/0.36837799819503636
    f8.label = 'f8'

    theta = beta1*f1 + beta2*f2 + beta3*f3 + beta4*f4 + beta5*f5 + beta6*f6 + beta7*f7 + beta8*f8; theta.label = 'theta'

    loading_at_zero_P = (
        -beta1*0.07469040214406462/0.233513075441677571 +
        -beta2*0.593439038924528/0.198559030356437852 + 
        -beta3*0.6258918477237851/0.199095730314402593 + 
        -beta4*0.026347531768028034/0.0330501158871211944 + 
        -beta4*0.451590663829706/0.49764971037256835 + 
        -beta6*0.04040880811128878/0.059921040583132836 + 
        -beta7*0.033471221601783376/0.112859511430337237 + 
        -beta8*0.3805783317820693/0.368377998195036368
    ); loading_at_zero_P.label = 'loading_at_zero_P'

    epsilon_max = Value(1.0, label='epsilon_max')
    d = (P - 0.0).abs(); d.label = 'd'
    alpha = Value(2.0, label='alpha')
    y_train_mean = Value(np.mean(y_train), label='y_train_mean')

    y = theta + intercept + y_train_mean + (loading_at_zero_P + y_train_mean).abs() * epsilon_max * (-alpha * d).exp()
    y.label = 'y'

    return y

In [ ]:
def predict_CH4_373K(
          P, density, spg, volume, SiOSi_average, SiOSi_gmean, SiOSi_hmean, SiOSi_max, SiOSi_mean, SiOSi_min,
          SiOSi_skew, SiOSi_std, SiOSi_var, SiO_average, SiO_gmean, SiO_hmean, SiO_max, SiO_mean, SiO_min,
          SiO_skew, SiO_std, SiO_var, ASA, AV, NASA, NAV, VolFrac, largest_free_sphere, largest_included_sphere,
          largest_included_sphere_free, max_dim, min_dim, mode_dim, pld, lcd, void_frac, surf_area_m2g, surf_area_m2cm3
      ):
      Value.reset()

      # input variables
      P = Value(P, label='P')
      density = Value(density, label='density')
      spg = Value(spg, label='spg')
      volume = Value(volume, label='volume')
      SiOSi_average = Value(SiOSi_average, label='SiOSi_average')
      SiOSi_gmean = Value(SiOSi_gmean, label='SiOSi_gmean')
      SiOSi_hmean = Value(SiOSi_hmean, label='SiOSi_hmean')
      SiOSi_max = Value(SiOSi_max, label='SiOSi_max')
      SiOSi_mean = Value(SiOSi_mean, label='SiOSi_mean')
      SiOSi_min = Value(SiOSi_min, label='SiOSi_min')
      SiOSi_skew = Value(SiOSi_skew, label='SiOSi_skew')
      SiOSi_std = Value(SiOSi_std, label='SiOSi_std')
      SiOSi_var = Value(SiOSi_var, label='SiOSi_var')
      SiO_average = Value(SiO_average, label='SiO_average')
      SiO_gmean = Value(SiO_gmean, label='SiO_gmean')
      SiO_hmean = Value(SiO_hmean, label='SiO_hmean')
      SiO_max = Value(SiO_max, label='SiO_max')
      SiO_mean = Value(SiO_mean, label='SiO_mean')
      SiO_min = Value(SiO_min, label='SiO_min')
      SiO_skew = Value(SiO_skew, label='SiO_skew')
      SiO_std = Value(SiO_std, label='SiO_std')
      SiO_var = Value(SiO_var, label='SiO_var')
      ASA = Value(ASA, label='ASA')
      AV = Value(AV, label='AV')
      NASA = Value(NASA, label='NASA')
      NAV = Value(NAV, label='NAV')
      VolFrac = Value(VolFrac, label='VolFrac')
      largest_free_sphere = Value(largest_free_sphere, label='largest_free_sphere')
      largest_included_sphere = Value(largest_included_sphere, label='largest_included_sphere')
      largest_included_sphere_free = Value(largest_included_sphere_free, label='largest_included_sphere_free')
      max_dim = Value(max_dim, label='max_dim')
      min_dim = Value(min_dim, label='min_dim')
      mode_dim = Value(mode_dim, label='mode_dim')
      pld = Value(pld, label='pld')
      lcd = Value(lcd, label='lcd')
      void_frac = Value(void_frac, label='void_frac')
      surf_area_m2g = Value(surf_area_m2g, label='surf_area_m2g')
      surf_area_m2cm3 = Value(surf_area_m2cm3, label='surf_area_m2cm3')

      # Constants
      beta1 = Value(0.1909906610367201, label='beta1')
      beta2 = Value(0.11682964613884445, label='beta2')
      beta3 = Value(-0.09344427307559947, label='beta3')
      beta4 = Value(0.20074867148999848, label='beta4')
      beta5 = Value(0.11122063059032497, label='beta5')
      beta6 = Value(0.5214533740342101, label='beta6')
      beta7 = Value(-0.3427553886500203, label='beta7')
      beta8 = Value(-0.21426185160030492, label='beta8')
      intercept = Value(0.0, label='intercept')

      # f terms
      f1 = ((1/1*1) * P / 
            (1 + (1/1*1) * P) - 
            0.5657765094736966)/0.1957614458698153
      f1.label = 'f1'

      f2 = (((-largest_included_sphere_free).exp()/1*surf_area_m2cm3) * P / 
            (1 + ((-largest_included_sphere_free).exp()/1*surf_area_m2cm3) * P) - 
            0.4894973861635249)/0.30511699644959617
      f2.label = 'f2'

      f3 = ((NAV.exp()/spg*1) * P / 
            (1 + (NAV.exp()/spg*1) * P) - 
            0.054277814589412984)/0.09756094243688461
      f3.label = 'f3'

      f4 = ((density.exp()/1*(VolFrac**2)) * P / 
            (1 + (density.exp()/1*(VolFrac**2)) * P) - 
            0.07338101892284041)/0.08803243033873245
      f4.label = 'f4'

      f5 = ((largest_included_sphere_free.exp()/(spg**2)/largest_free_sphere.exp()) * P / 
            (1 + (largest_included_sphere_free.exp()/(spg**2)/largest_free_sphere.exp()) * P) - 
            0.04457244307054062)/0.14950551357237374
      f5.label = 'f5'

      f6 = ((void_frac**2/1/1) * P / 
            (1 + (void_frac**2/1/1) * P) - 
            0.13314539459770083)/0.09567259821436203
      f6.label = 'f6'

      f7 = ((VolFrac**2/1/void_frac**2) * P / 
            (1 + (VolFrac**2/1/void_frac**2) * P) - 
            0.11368892880920235)/0.1094460306216634
      f7.label = 'f7'

      f8 = (((-surf_area_m2g).exp()*1*1) * P / 
            (1 + ((-surf_area_m2g).exp()*1*1) * P) - 
            0.0539764858809039)/0.17390362993878106
      f8.label = 'f8'

      theta = beta1*f1 + beta2*f2 + beta3*f3 + beta4*f4 + beta5*f5 + beta6*f6 + beta7*f7 + beta8*f8; theta.label = 'theta'

      loading_at_zero_P = (
            -beta1*0.5657765094736966/0.19576144586981531 +
            -beta2*0.4894973861635249/0.305116996449596172 + 
            -beta3*0.054277814589412984/0.097560942436884613 + 
            -beta4*0.07338101892284041/0.088032430338732454 + 
            -beta5*0.04457244307054062/0.149505513572373745 + 
            -beta6*0.13314539459770083/0.095672598214362036 + 
            -beta7*0.11368892880920235/0.10944603062166347 + 
            -beta8*0.0539764858809039/0.173903629938781068
      ); loading_at_zero_P.label = 'loading_at_zero_P'

      epsilon_max = Value(1.0, label='epsilon_max')
      d = (P - 0.0).abs(); d.label = 'd'
      alpha = Value(2.0, label='alpha')
      y_train_mean = Value(np.mean(y_train), label='y_train_mean')

      y = theta + intercept + y_train_mean - (loading_at_zero_P + y_train_mean).abs() * epsilon_max * (-alpha * d).exp()
      y.label = 'y'

      return y

### CO2 Equations

In [ ]:
def predict_CO2_323K(
          P, density, spg, volume, SiOSi_average, SiOSi_gmean, SiOSi_hmean, SiOSi_max, SiOSi_mean, SiOSi_min,
          SiOSi_skew, SiOSi_std, SiOSi_var, SiO_average, SiO_gmean, SiO_hmean, SiO_max, SiO_mean, SiO_min,
          SiO_skew, SiO_std, SiO_var, ASA, AV, NASA, NAV, VolFrac, largest_free_sphere, largest_included_sphere,
          largest_included_sphere_free, max_dim, min_dim, mode_dim, pld, lcd, void_frac, surf_area_m2g, surf_area_m2cm3
      ):
      Value.reset()

      # input variables
      P = Value(P, label='P')
      density = Value(density, label='density')
      spg = Value(spg, label='spg')
      volume = Value(volume, label='volume')
      SiOSi_average = Value(SiOSi_average, label='SiOSi_average')
      SiOSi_gmean = Value(SiOSi_gmean, label='SiOSi_gmean')
      SiOSi_hmean = Value(SiOSi_hmean, label='SiOSi_hmean')
      SiOSi_max = Value(SiOSi_max, label='SiOSi_max')
      SiOSi_mean = Value(SiOSi_mean, label='SiOSi_mean')
      SiOSi_min = Value(SiOSi_min, label='SiOSi_min')
      SiOSi_skew = Value(SiOSi_skew, label='SiOSi_skew')
      SiOSi_std = Value(SiOSi_std, label='SiOSi_std')
      SiOSi_var = Value(SiOSi_var, label='SiOSi_var')
      SiO_average = Value(SiO_average, label='SiO_average')
      SiO_gmean = Value(SiO_gmean, label='SiO_gmean')
      SiO_hmean = Value(SiO_hmean, label='SiO_hmean')
      SiO_max = Value(SiO_max, label='SiO_max')
      SiO_mean = Value(SiO_mean, label='SiO_mean')
      SiO_min = Value(SiO_min, label='SiO_min')
      SiO_skew = Value(SiO_skew, label='SiO_skew')
      SiO_std = Value(SiO_std, label='SiO_std')
      SiO_var = Value(SiO_var, label='SiO_var')
      ASA = Value(ASA, label='ASA')
      AV = Value(AV, label='AV')
      NASA = Value(NASA, label='NASA')
      NAV = Value(NAV, label='NAV')
      VolFrac = Value(VolFrac, label='VolFrac')
      largest_free_sphere = Value(largest_free_sphere, label='largest_free_sphere')
      largest_included_sphere = Value(largest_included_sphere, label='largest_included_sphere')
      largest_included_sphere_free = Value(largest_included_sphere_free, label='largest_included_sphere_free')
      max_dim = Value(max_dim, label='max_dim')
      min_dim = Value(min_dim, label='min_dim')
      mode_dim = Value(mode_dim, label='mode_dim')
      pld = Value(pld, label='pld')
      lcd = Value(lcd, label='lcd')
      void_frac = Value(void_frac, label='void_frac')
      surf_area_m2g = Value(surf_area_m2g, label='surf_area_m2g')
      surf_area_m2cm3 = Value(surf_area_m2cm3, label='surf_area_m2cm3')

      # Constants
      beta1 = Value(0.2408158535926262, label='beta1')
      beta2 = Value(-0.6051468741071055, label='beta2')
      beta3 = Value(0.17771546567926963, label='beta3')
      beta4 = Value(-2.0804716603104425, label='beta4')
      beta5 = Value(-0.35101203615301246, label='beta5')
      beta6 = Value(1.1201332339583743, label='beta6')
      beta7 = Value(0.0023376143648303396, label='beta7')
      beta8 = Value(-0.12134179550190091, label='beta8')
      beta9 = Value(-1.215940666467515, label='beta9')
      beta10 = Value(2.3657034591203896, label='beta10')
      beta11 = Value(-0.6619785317203183, label='beta11')
      beta12 = Value(1.712890108697996, label='beta12')
      intercept = Value(0.0, label='intercept')

      # f terms
      f1 = ((NASA*volume/1) * P / 
            (1 + (NASA*volume/1) * P) - 
            0.5053560148639079)/0.4960635647959033
      f1.label = 'f1'

      f2 = ((NAV/largest_free_sphere.exp()*(largest_included_sphere**2)) * P / 
            (1 + (NAV/largest_free_sphere.exp()*(largest_included_sphere**2)) * P) - 
            0.022023073277147264)/0.07412726445549916
      f2.label = 'f2'

      f3 = ((spg.exp()*(-NASA).exp()/AV.exp()) * P / 
            (1 + (spg.exp()*(-NASA).exp()/AV.exp()) * P) - 
            0.636492475959887)/0.4788319028018178
      f3.label = 'f3'

      f4 = ((density*VolFrac*1) * P / 
            (1 + (density*VolFrac*1) * P) - 
            0.15658870051279075)/0.13014735733464328
      f4.label = 'f4'

      f5 = ((lcd**2/1/largest_free_sphere.exp()) * P / 
            (1 + (lcd**2/1/largest_free_sphere.exp()) * P) - 
            0.4486063640488222)/0.2755565010621821
      f5.label = 'f5'

      f6 = ((volume/density.exp()/VolFrac.exp()) * P / 
            (1 + (volume/density.exp()/VolFrac.exp()) * P) - 
            0.769527662996705)/0.1914254994515539
      f6.label = 'f6'

      f7 = ((NASA/spg.exp()/1) * P / 
            (1 + (NASA/spg.exp()/1) * P) - 
            0.018544154536591098)/0.1134031399766316
      f7.label = 'f7'

      f8 = ((lcd**2/1/1) * P / 
            (1 + (lcd**2/1/1) * P) - 
            0.9474631921424108)/0.11185899780233935
      f8.label = 'f8'

      f9 = ((pld/volume*1) * P / 
            (1 + (pld/volume*1) * P) - 
            0.2259632424704171)/0.12819776041479525
      f9.label = 'f9'

      f10 = ((void_frac/1/1) * P / 
            (1 + (void_frac/1/1) * P) - 
            0.2983059470851118)/0.15314334975683067
      f10.label = 'f10'

      f11 = ((1*1*1) * P / 
            (1 + (1*1*1) * P) - 
            0.561041654561111)/0.19904288999498684
      f11.label = 'f11'

      f12 = ((VolFrac*1*1) * P / 
            (1 + (VolFrac*1*1) * P) - 
            0.10745167163360558)/0.09939185680979355
      f12.label = 'f12'

      theta = (
            beta1*f1 + beta2*f2 + beta3*f3 + beta4*f4 + beta5*f5 + beta6*f6 + beta7*f7 + beta8*f8 + beta9*f9 + beta10*f10 + beta11*f11 + beta12*f12
      ); theta.label = 'theta'

      loading_at_zero_P = (
            -beta1*0.5053560148639079/0.49606356479590331 + 
            -beta2*0.022023073277147264/0.074127264455499162 + 
            -beta3*0.636492475959887/0.47883190280181783 + 
            -beta4*0.15658870051279075/0.130147357334643284 + 
            -beta5*0.4486063640488222/0.27555650106218215 + 
            -beta6*0.769527662996705/0.19142549945155396 + 
            -beta7*0.018544154536591098/0.11340313997663167 + 
            -beta8*0.9474631921424108/0.111858997802339358 + 
            -beta9*0.2259632424704171/0.128197760414795259 + 
            -beta10*0.2983059470851118/0.1531433497568306710 + 
            -beta11*0.561041654561111/0.1990428899949868411 + 
            -beta12*0.10745167163360558/0.0993918568097935512
      ); loading_at_zero_P.label = 'loading_at_zero_P'

      epsilon_max = Value(1.0, label='epsilon_max')
      d = (P - 0.0).abs(); d.label = 'd'
      alpha = Value(2.0, label='alpha')
      y_train_mean = Value(np.mean(y_train), label='y_train_mean')

      y = theta + intercept + y_train_mean - (loading_at_zero_P + y_train_mean).abs() * epsilon_max * (-alpha * d).exp()
      y.label = 'y'

      return y

In [ ]:
def predict_CO2_373K(
        P, density, spg, volume, SiOSi_average, SiOSi_gmean, SiOSi_hmean, SiOSi_max, SiOSi_mean, SiOSi_min,
        SiOSi_skew, SiOSi_std, SiOSi_var, SiO_average, SiO_gmean, SiO_hmean, SiO_max, SiO_mean, SiO_min,
        SiO_skew, SiO_std, SiO_var, ASA, AV, NASA, NAV, VolFrac, largest_free_sphere, largest_included_sphere,
        largest_included_sphere_free, max_dim, min_dim, mode_dim, pld, lcd, void_frac, surf_area_m2g, surf_area_m2cm3
    ):
    Value.reset()

    # input variables
    P = Value(P, label='P')
    density = Value(density, label='density')
    spg = Value(spg, label='spg')
    volume = Value(volume, label='volume')
    SiOSi_average = Value(SiOSi_average, label='SiOSi_average')
    SiOSi_gmean = Value(SiOSi_gmean, label='SiOSi_gmean')
    SiOSi_hmean = Value(SiOSi_hmean, label='SiOSi_hmean')
    SiOSi_max = Value(SiOSi_max, label='SiOSi_max')
    SiOSi_mean = Value(SiOSi_mean, label='SiOSi_mean')
    SiOSi_min = Value(SiOSi_min, label='SiOSi_min')
    SiOSi_skew = Value(SiOSi_skew, label='SiOSi_skew')
    SiOSi_std = Value(SiOSi_std, label='SiOSi_std')
    SiOSi_var = Value(SiOSi_var, label='SiOSi_var')
    SiO_average = Value(SiO_average, label='SiO_average')
    SiO_gmean = Value(SiO_gmean, label='SiO_gmean')
    SiO_hmean = Value(SiO_hmean, label='SiO_hmean')
    SiO_max = Value(SiO_max, label='SiO_max')
    SiO_mean = Value(SiO_mean, label='SiO_mean')
    SiO_min = Value(SiO_min, label='SiO_min')
    SiO_skew = Value(SiO_skew, label='SiO_skew')
    SiO_std = Value(SiO_std, label='SiO_std')
    SiO_var = Value(SiO_var, label='SiO_var')
    ASA = Value(ASA, label='ASA')
    AV = Value(AV, label='AV')
    NASA = Value(NASA, label='NASA')
    NAV = Value(NAV, label='NAV')
    VolFrac = Value(VolFrac, label='VolFrac')
    largest_free_sphere = Value(largest_free_sphere, label='largest_free_sphere')
    largest_included_sphere = Value(largest_included_sphere, label='largest_included_sphere')
    largest_included_sphere_free = Value(largest_included_sphere_free, label='largest_included_sphere_free')
    max_dim = Value(max_dim, label='max_dim')
    min_dim = Value(min_dim, label='min_dim')
    mode_dim = Value(mode_dim, label='mode_dim')
    pld = Value(pld, label='pld')
    lcd = Value(lcd, label='lcd')
    void_frac = Value(void_frac, label='void_frac')
    surf_area_m2g = Value(surf_area_m2g, label='surf_area_m2g')
    surf_area_m2cm3 = Value(surf_area_m2cm3, label='surf_area_m2cm3')

    # Constants
    beta1 = Value(-0.679081742692596, label='beta1')
    beta2 = Value(0.20307777789750459, label='beta2')
    beta3 = Value(0.4329129291782464, label='beta3')
    beta4 = Value(0.3464949739458949, label='beta4')
    beta5 = Value(0.17098884194225522, label='beta5')
    beta6 = Value(2.2905076502184967, label='beta6')
    beta7 = Value(-0.22028767452266362, label='beta7')
    beta8 = Value(-1.2269155949581563, label='beta8')
    beta9 = Value(-0.8108018121501624, label='beta9')
    beta10 = Value(0.05328000691618266, label='beta10')
    beta11 = Value(-0.19349635098380905, label='beta11')
    beta12 = Value(0.284054306504558, label='beta12')
    intercept = Value(0.0, label='intercept')

    # f terms
    f1 = (((-pld).exp()/1/1) * P / 
          (1 + ((-pld).exp()/1/1) * P) - 
          0.06471130620155797)/0.08312215260703829
    f1.label = 'f1'

    f2 = (((-surf_area_m2g).exp()*1*NASA) * P / 
          (1 + ((-surf_area_m2g).exp()*1*NASA) * P) - 
          0.047511004998993515)/0.20078477954126026
    f2.label = 'f2'

    f3 = (((-largest_included_sphere).exp()*(-NAV).exp()/1) * P / 
          (1 + ((-largest_included_sphere).exp()*(-NAV).exp()/1) * P) - 
          0.011389997203239096)/0.018491549386222977
    f3.label = 'f3'

    f4 = (((-lcd).exp()*(volume**2)*1) * P / 
          (1 + ((-lcd).exp()*(volume**2)*1) * P) - 
          0.5646842961313753)/0.30068050566097687
    f4.label = 'f4'

    f5 = ((void_frac.exp()/density*1) * P / 
          (1 + (void_frac.exp()/density*1) * P) - 
          0.5136090001227659)/0.19590195356823578
    f5.label = 'f5'

    f6 = (((-NAV).exp()*void_frac*1) * P / 
          (1 + ((-NAV).exp()*void_frac*1) * P) - 
          0.3000617637006874)/0.15336443564059765
    f6.label = 'f6'

    f7 = (((-void_frac).exp()/1*1) * P / 
          (1 + ((-void_frac).exp()/1*1) * P) - 
          0.49975562767787235)/0.191365996679738
    f7.label = 'f7'

    f8 = (((density**2)*VolFrac*VolFrac.exp()) * P / 
          (1 + ((density**2)*VolFrac*VolFrac.exp()) * P) - 
          0.24088977178365023)/0.18102771146462537
    f8.label = 'f8'

    f9 = ((largest_free_sphere/1/(largest_included_sphere_free**2)) * P / 
          (1 + (largest_free_sphere/1/(largest_included_sphere_free**2)) * P) - 
          0.16738596411370837)/0.10655406891164021
    f9.label = 'f9'

    f10 = (((NASA**2)/1/NASA.exp()) * P / 
           (1 + ((NASA**2)/1/NASA.exp()) * P) - 
           0.001524646748204257)/0.014373101141606134
    f10.label = 'f10'

    f11 = ((surf_area_m2g*(NAV**2)*1) * P / 
           (1 + (surf_area_m2g*(NAV**2)*1) * P) - 
           0.030270422267265115)/0.1039627226006907
    f11.label = 'f11'

    f12 = ((VolFrac/largest_free_sphere*1) * P / 
           (1 + (VolFrac/largest_free_sphere*1) * P) - 
           0.029136294307661747)/0.027715446590296355
    f12.label = 'f12'

    theta = (beta1*f1 + beta2*f2 + beta3*f3 + beta4*f4 + beta5*f5 + beta6*f6 + 
            beta7*f7 + beta8*f8 + beta9*f9 + beta10*f10 + beta11*f11 + beta12*f12); theta.label = 'theta'

    loading_at_zero_P = (
        -beta1*0.06471130620155797/0.083122152607038291 + 
        -beta2*0.047511004998993515/0.200784779541260262 + 
        -beta3*0.011389997203239096/0.0184915493862229773 + 
        -beta4*0.5646842961313753/0.300680505660976874 + 
        -beta5*0.5136090001227659/0.195901953568235785 + 
        -beta6*0.3000617637006874/0.153364435640597656 + 
        -beta7*0.49975562767787235/0.1913659966797387 + 
        -beta8*0.24088977178365023/0.181027711464625378 + 
        -beta9*0.16738596411370837/0.106554068911640219 + 
        -beta10*0.001524646748204257/0.01437310114160613410 + 
        -beta11*0.030270422267265115/0.103962722600690711 + 
        -beta12*0.029136294307661747/0.02771544659029635512
    ); loading_at_zero_P.label = 'loading_at_zero_P'

    epsilon_max = Value(1.0, label='epsilon_max')
    d = (P - 0.0).abs(); d.label = 'd'
    alpha = Value(2.0, label='alpha')
    y_train_mean = Value(np.mean(y_train), label='y_train_mean')

    y = theta + intercept + y_train_mean - (loading_at_zero_P + y_train_mean).abs() * epsilon_max * (-alpha * d).exp()
    y.label = 'y'

    return y


### N2 Equations

In [ ]:
def predict_N2_323K(
        P, density, spg, volume, SiOSi_average, SiOSi_gmean, SiOSi_hmean, SiOSi_max, SiOSi_mean, SiOSi_min,
        SiOSi_skew, SiOSi_std, SiOSi_var, SiO_average, SiO_gmean, SiO_hmean, SiO_max, SiO_mean, SiO_min,
        SiO_skew, SiO_std, SiO_var, ASA, AV, NASA, NAV, VolFrac, largest_free_sphere, largest_included_sphere,
        largest_included_sphere_free, max_dim, min_dim, mode_dim, pld, lcd, void_frac, surf_area_m2g, surf_area_m2cm3
    ):
    Value.reset()

    # input variables
    P = Value(P, label='P')
    density = Value(density, label='density')
    spg = Value(spg, label='spg')
    volume = Value(volume, label='volume')
    SiOSi_average = Value(SiOSi_average, label='SiOSi_average')
    SiOSi_gmean = Value(SiOSi_gmean, label='SiOSi_gmean')
    SiOSi_hmean = Value(SiOSi_hmean, label='SiOSi_hmean')
    SiOSi_max = Value(SiOSi_max, label='SiOSi_max')
    SiOSi_mean = Value(SiOSi_mean, label='SiOSi_mean')
    SiOSi_min = Value(SiOSi_min, label='SiOSi_min')
    SiOSi_skew = Value(SiOSi_skew, label='SiOSi_skew')
    SiOSi_std = Value(SiOSi_std, label='SiOSi_std')
    SiOSi_var = Value(SiOSi_var, label='SiOSi_var')
    SiO_average = Value(SiO_average, label='SiO_average')
    SiO_gmean = Value(SiO_gmean, label='SiO_gmean')
    SiO_hmean = Value(SiO_hmean, label='SiO_hmean')
    SiO_max = Value(SiO_max, label='SiO_max')
    SiO_mean = Value(SiO_mean, label='SiO_mean')
    SiO_min = Value(SiO_min, label='SiO_min')
    SiO_skew = Value(SiO_skew, label='SiO_skew')
    SiO_std = Value(SiO_std, label='SiO_std')
    SiO_var = Value(SiO_var, label='SiO_var')
    ASA = Value(ASA, label='ASA')
    AV = Value(AV, label='AV')
    NASA = Value(NASA, label='NASA')
    NAV = Value(NAV, label='NAV')
    VolFrac = Value(VolFrac, label='VolFrac')
    largest_free_sphere = Value(largest_free_sphere, label='largest_free_sphere')
    largest_included_sphere = Value(largest_included_sphere, label='largest_included_sphere')
    largest_included_sphere_free = Value(largest_included_sphere_free, label='largest_included_sphere_free')
    max_dim = Value(max_dim, label='max_dim')
    min_dim = Value(min_dim, label='min_dim')
    mode_dim = Value(mode_dim, label='mode_dim')
    pld = Value(pld, label='pld')
    lcd = Value(lcd, label='lcd')
    void_frac = Value(void_frac, label='void_frac')
    surf_area_m2g = Value(surf_area_m2g, label='surf_area_m2g')
    surf_area_m2cm3 = Value(surf_area_m2cm3, label='surf_area_m2cm3')

    # Constants
    beta1 = Value(-0.06012649624795871, label='beta1')
    beta2 = Value(0.29171999178744773, label='beta2')
    beta3 = Value(-0.001505369599999802, label='beta3')
    beta4 = Value(0.030830007929509663, label='beta4')
    beta5 = Value(-0.3257118942142511, label='beta5')
    beta6 = Value(0.5292953747226462, label='beta6')
    intercept = Value(0.0, label='intercept')

    # f terms
    f1 = ((pld**2*void_frac/volume**2) * P / 
          (1 + (pld**2*void_frac/volume**2) * P) - 
          0.024842285545539616)/0.029315086987157343
    f1.label = 'f1'

    f2 = ((void_frac*lcd/density**2) * P / 
          (1 + (void_frac*lcd/density**2) * P) - 
          0.4555766667320463)/0.22906966157063888
    f2.label = 'f2'

    f3 = ((1*void_frac**2*spg) * P / 
          (1 + (1*void_frac**2*spg) * P) - 
          0.7755018982379761)/0.2409826657271743
    f3.label = 'f3'

    f4 = ((NASA/density**2/1) * P / 
          (1 + (NASA/density**2/1) * P) - 
          0.48063444011241646)/0.4778963075261082
    f4.label = 'f4'

    f5 = ((largest_included_sphere_free*lcd*VolFrac**2) * P / 
          (1 + (largest_included_sphere_free*lcd*VolFrac**2) * P) - 
          0.3032790874679304)/0.30172130410084774
    f5.label = 'f5'

    f6 = ((1/1*void_frac**2) * P / 
          (1 + (1/1*void_frac**2) * P) - 
          0.13327729792619633)/0.09726926695513827
    f6.label = 'f6'

    theta = beta1*f1 + beta2*f2 + beta3*f3 + beta4*f4 + beta5*f5 + beta6*f6; theta.label = 'theta'

    loading_at_zero_P = (
        -beta1*0.024842285545539616/0.0293150869871573431 + 
        -beta2*0.4555766667320463/0.229069661570638882 + 
        -beta3*0.7755018982379761/0.24098266572717433 + 
        -beta4*0.48063444011241646/0.47789630752610824 + 
        -beta5*0.3032790874679304/0.301721304100847745 + 
        -beta6*0.13327729792619633/0.097269266955138276  
    ); loading_at_zero_P.label = 'loading_at_zero_P'

    epsilon_max = Value(1.0, label='epsilon_max')
    d = (P - 0.0).abs(); d.label = 'd'
    alpha = Value(2.0, label='alpha')
    y_train_mean = Value(np.mean(y_train), label='y_train_mean')

    y = theta + intercept + y_train_mean - (loading_at_zero_P + y_train_mean).abs() * epsilon_max * (-alpha * d).exp()
    y.label = 'y'

    return y

In [ ]:
def predict_N2_373K(
            P, density, spg, volume, SiOSi_average, SiOSi_gmean, SiOSi_hmean, SiOSi_max, SiOSi_mean, SiOSi_min,
            SiOSi_skew, SiOSi_std, SiOSi_var, SiO_average, SiO_gmean, SiO_hmean, SiO_max, SiO_mean, SiO_min,
            SiO_skew, SiO_std, SiO_var, ASA, AV, NASA, NAV, VolFrac, largest_free_sphere, largest_included_sphere,
            largest_included_sphere_free, max_dim, min_dim, mode_dim, pld, lcd, void_frac, surf_area_m2g, surf_area_m2cm3
      ):
      Value.reset()

      # input variables
      P = Value(P, label='P')
      density = Value(density, label='density')
      spg = Value(spg, label='spg')
      volume = Value(volume, label='volume')
      SiOSi_average = Value(SiOSi_average, label='SiOSi_average')
      SiOSi_gmean = Value(SiOSi_gmean, label='SiOSi_gmean')
      SiOSi_hmean = Value(SiOSi_hmean, label='SiOSi_hmean')
      SiOSi_max = Value(SiOSi_max, label='SiOSi_max')
      SiOSi_mean = Value(SiOSi_mean, label='SiOSi_mean')
      SiOSi_min = Value(SiOSi_min, label='SiOSi_min')
      SiOSi_skew = Value(SiOSi_skew, label='SiOSi_skew')
      SiOSi_std = Value(SiOSi_std, label='SiOSi_std')
      SiOSi_var = Value(SiOSi_var, label='SiOSi_var')
      SiO_average = Value(SiO_average, label='SiO_average')
      SiO_gmean = Value(SiO_gmean, label='SiO_gmean')
      SiO_hmean = Value(SiO_hmean, label='SiO_hmean')
      SiO_max = Value(SiO_max, label='SiO_max')
      SiO_mean = Value(SiO_mean, label='SiO_mean')
      SiO_min = Value(SiO_min, label='SiO_min')
      SiO_skew = Value(SiO_skew, label='SiO_skew')
      SiO_std = Value(SiO_std, label='SiO_std')
      SiO_var = Value(SiO_var, label='SiO_var')
      ASA = Value(ASA, label='ASA')
      AV = Value(AV, label='AV')
      NASA = Value(NASA, label='NASA')
      NAV = Value(NAV, label='NAV')
      VolFrac = Value(VolFrac, label='VolFrac')
      largest_free_sphere = Value(largest_free_sphere, label='largest_free_sphere')
      largest_included_sphere = Value(largest_included_sphere, label='largest_included_sphere')
      largest_included_sphere_free = Value(largest_included_sphere_free, label='largest_included_sphere_free')
      max_dim = Value(max_dim, label='max_dim')
      min_dim = Value(min_dim, label='min_dim')
      mode_dim = Value(mode_dim, label='mode_dim')
      pld = Value(pld, label='pld')
      lcd = Value(lcd, label='lcd')
      void_frac = Value(void_frac, label='void_frac')
      surf_area_m2g = Value(surf_area_m2g, label='surf_area_m2g')
      surf_area_m2cm3 = Value(surf_area_m2cm3, label='surf_area_m2cm3')

      # Constants
      beta1 = Value(0.07337148018041849, label='beta1')
      beta2 = Value(0.47, label='beta2')
      beta3 = Value(-0.2230577953955047, label='beta3')
      beta4 = Value(0.0414632001791709, label='beta4')
      beta5 = Value(0.043098439630175324, label='beta5')
      intercept = Value(0.0, label='intercept')

      # f terms
      f1 = ((1/density**2/1) * P / 
            (1 + (1/density**2/1) * P) - 
            0.34510445568068854)/0.15844603241860306
      f1.label = 'f1'

      f2 = ((1*1*void_frac**2) * P / 
            (1 + (1*1*void_frac**2) * P) - 
            0.13402139615626685)/0.09807949467217342
      f2.label = 'f2'

      f3 = ((VolFrac**2*1*volume) * P / 
            (1 + (VolFrac**2*1*volume) * P) - 
            0.2099562104652133)/0.2192665701701981
      f3.label = 'f3'

      f4 = ((ASA*1*VolFrac**2) * P / 
            (1 + (ASA*1*VolFrac**2) * P) - 
            0.43892114771426277)/0.43306015276923676
      f4.label = 'f4'

      f5 = ((NASA*largest_included_sphere/volume**2) * P / 
            (1 + (NASA*largest_included_sphere/volume**2) * P) - 
            0.3362313626509776)/0.387612317674983
      f5.label = 'f5'

      theta = beta1*f1 + beta2*f2 + beta3*f3 + beta4*f4 + beta5*f5; theta.label = 'theta'

      loading_at_zero_P = (
            -beta1*0.34510445568068854/0.158446032418603061 + 
            -beta2*0.13402139615626685/0.098079494672173422 + 
            -beta3*0.2099562104652133/0.21926657017019813 + 
            -beta4*0.43892114771426277/0.433060152769236765 + 
            -beta5*0.3362313626509776/0.3876123176749836
      ); loading_at_zero_P.label = 'loading_at_zero_P'

      epsilon_max = Value(1.0, label='epsilon_max')
      d = (P - 0.0).abs(); d.label = 'd'
      alpha = Value(2.0, label='alpha')
      y_train_mean = Value(np.mean(y_train), label='y_train_mean')

      y = theta + intercept + y_train_mean + (loading_at_zero_P + y_train_mean).abs() * epsilon_max * (-alpha * d).exp()
      y.label = 'y'

      return y

### Function Map

In [ ]:
function_map_AI_DARWIN = {
    'CH4': {
        323: predict_CH4_323K,
        373: predict_CH4_373K,
    },
    'CO2': {
        323: predict_CO2_323K,
        373: predict_CO2_373K,
    },
    'N2': {
        323: predict_N2_323K,
        373: predict_N2_373K,
    }
}
AI_DARWIN_EQUATION = function_map_AI_DARWIN[ADSORBATE][TEMPERATURE]

## Generate and Prepare Data

In [ ]:
zeolite_names = []
for zeolite_name, df in ADSORBATE_DATA.items():
    zeolite_names.append(zeolite_name)
    # Rename columns to our variable names
    df.rename(columns={'Temperature (K)': 'temperature', 'Pressure (Pa)' : 'P'}, inplace=True)

    # Divide pressure term by P_TRAIN_STD to follow Arijit's methodology to generate the AI DARWIN equation
    df['P'] = df['P'] / P_TRAIN_STD

    # Split each sheet into temperature buckets
    df_grouped_by_temperature = df.groupby('temperature')
    ADSORBATE_DATA[zeolite_name] = {temperature: df_group for temperature, df_group in df_grouped_by_temperature}

ADSORBATE_DATA[FOREGROUND_ADSORBENT][TEMPERATURE]

In [ ]:
# Remove every row that contains a nan
ZEOLITE_DATA = _ZEOLITE_DATA.dropna(inplace=False)

# Drop unnamed columns
ZEOLITE_DATA = ZEOLITE_DATA.loc[:, ~ZEOLITE_DATA.columns.str.contains('^Unnamed')]

# Convert ZEOLITE_DATA into a dictionary with zeolite name as the key and the values are the properties of the zeolite in dict format
ZEOLITE_DATA = ZEOLITE_DATA.set_index('name').to_dict(orient='index')
ZEOLITE_DATA[FOREGROUND_ADSORBENT]

In [ ]:
common_zeolites = set()
missing_zeolites = set()
# Add zeolite properties to each adsorbate sheet
for zeolite_name, adsorbate_grouped_by_temperature in ADSORBATE_DATA.items():
    for temperature, df in adsorbate_grouped_by_temperature.items():
        try:
            for zeolite_property in ZEOLITE_DATA[zeolite_name]:
                common_zeolites.add(zeolite_name)
                ADSORBATE_DATA[zeolite_name][temperature][zeolite_property] = ZEOLITE_DATA[zeolite_name][zeolite_property]
        except KeyError: # Case where we are missing data on a given zeolite
            missing_zeolites.add(zeolite_name)

# Delete all adsorbate sheets that use a zeolite with missing data:
for zeolite_name in missing_zeolites:
    del ADSORBATE_DATA[zeolite_name]

ADSORBATE_DATA[FOREGROUND_ADSORBENT][TEMPERATURE]

In [ ]:
print("Missing zeolites:", list(missing_zeolites))
print("Common zeolites:", list(common_zeolites))

In [ ]:
len(common_zeolites)

In [ ]:
# TODO: figure out how to suppress warnings

# Populate the AI-DARWIN eq for each catalyst at the given temperature
for catalyst_name, adsorbate_grouped_by_temperature in ADSORBATE_DATA.items():
    for temperature, df in adsorbate_grouped_by_temperature.items():
        if temperature == TEMPERATURE:
            populate_dataframe(df, AI_DARWIN_EQUATION)

In [ ]:
_X = ADSORBATE_DATA[FOREGROUND_ADSORBENT][TEMPERATURE]
_X

In [ ]:
# Plot the P on the x-axis and Absolute Loading (mol/kg) and y on the y-axis
_X.plot(x='P', y=['Absolute Loading (mol/kg)', 'y'], kind='line')

## Generate Computational Graph

In [ ]:
Value.reset()
# Pass in arbitrary values into our function because we are just trying to construct the causal graph
y = AI_DARWIN_EQUATION(*([0.1] * len(inspect.signature(AI_DARWIN_EQUATION).parameters)))
target_name = 'y'

computational_graph = draw_computational_graph(y, disable_data_field=True, disable_grad_field=True)
print("Computational Graph Generated")
computational_graph

## Build Causal Graph

In [ ]:
X = _X.drop(labels=['y'], axis=1)

causal_graph = build_causal_graph(y, X, target_name, debug=False)
causal_graph.draw(rankdir='LR')

## Shapley Flow Analysis

### Holding Pressure Constant

In [ ]:
nruns = 100 # number of monte carlo samples
sample_ind = 0
bg = ADSORBATE_DATA[BACKGROUND_ADSORBENT][TEMPERATURE].iloc[[PRESSURE_IDX]].drop(labels=['y'], axis=1)
fg = X.iloc[[PRESSURE_IDX]] # foreground samples to explain. Selecting row from X such that the pressure value is equal to the pressure value of bg
bg = bg[fg.columns] # ensure fg and bg have same column order. NOTE: this only works when there is 1 fg

In [ ]:
print(BACKGROUND_ADSORBENT)
bg

In [ ]:
print(FOREGROUND_ADSORBENT)
fg

In [ ]:
cf_flow = GraphExplainer(causal_graph, bg, nruns=nruns).shap_values(fg, rankdir='LR')
cf_flow.fold_noise = True
shapflow_graph_with_constant_pressure = cf_flow.draw(sample_ind, show_fg_val=False)#, rankdir='TB')

### Simplified Shapley Flow with f Terms Only

#### Causal Link Functions

##### CH4

In [ ]:
def get_causal_links_with_f_terms_only_CH4_323K():
    # Constants
    beta1 = -0.4054507842872319
    beta2 = -3.5824569383000946
    beta3 = 3.9932667095050705
    beta4 = 0.5222507180877926
    beta5 = 0.004795464592376811
    beta6 = -0.1129628499436825
    beta7 = 0.013835556053187287
    beta8 = -0.09608874134668977
    betas = [beta1, beta2, beta3, beta4, beta5, beta6, beta7, beta8]
    intercept = 0.0
    loading_at_zero_P = (
        -beta1*0.07469040214406462/0.233513075441677571 +
        -beta2*0.593439038924528/0.198559030356437852 + 
        -beta3*0.6258918477237851/0.199095730314402593 + 
        -beta4*0.026347531768028034/0.0330501158871211944 + 
        -beta5*0.451590663829706/0.49764971037256835 + 
        -beta6*0.04040880811128878/0.059921040583132836 + 
        -beta7*0.033471221601783376/0.112859511430337237 + 
        -beta8*0.3805783317820693/0.368377998195036368
    )
    epsilon_max = 1.0
    y_train_mean = np.mean(y_train)
    alpha = 2.0

    causal_links = CausalLinks()
    target_name = 'y'
    X_with_only_f_terms = X[list(inspect.signature(AI_DARWIN_EQUATION).parameters.keys()) + ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8']]

    causal_links.add_causes_effects(
    ['largest_included_sphere', 'NAV', 'surf_area_m2cm3', 'P'],
    ['f1'],
    lambda largest_included_sphere, NAV, surf_area_m2cm3, P: 
        ((largest_included_sphere * np.exp(NAV) * np.exp(-surf_area_m2cm3)) * P / 
        (1 + (largest_included_sphere * np.exp(NAV) * np.exp(-surf_area_m2cm3)) * P) - 
        0.07469040214406462) / 0.23351307544167757
    )
    causal_links.add_causes_effects(
    ['VolFrac', 'AV', 'P'],
    ['f2'],
    lambda VolFrac, AV, P: 
        ((np.exp(VolFrac) / np.exp(-AV)) * P / 
        (1 + (np.exp(VolFrac) / np.exp(-AV)) * P) - 
        0.593439038924528) / 0.19855903035643785
    )
    causal_links.add_causes_effects(
    ['void_frac', 'P'],
    ['f3'],
    lambda void_frac, P: 
        ((np.exp(void_frac)) * P / 
        (1 + (np.exp(void_frac)) * P) - 
        0.6258918477237851) / 0.19909573031440259
    )
    causal_links.add_causes_effects(
    ['AV', 'VolFrac', 'void_frac', 'P'],
    ['f4'],
    lambda AV, VolFrac, void_frac, P: 
        ((AV / np.exp(VolFrac) * void_frac) * P / 
        (1 + (AV / np.exp(VolFrac) * void_frac) * P) - 
        0.026347531768028034) / 0.033050115887121194
    )
    causal_links.add_causes_effects(
    ['NASA', 'surf_area_m2cm3', 'spg', 'P'],
    ['f5'],
    lambda NASA, surf_area_m2cm3, spg, P: 
        ((np.exp(-NASA) * (surf_area_m2cm3**2) * spg) * P / 
        (1 + (np.exp(-NASA) * (surf_area_m2cm3**2) * spg) * P) - 
        0.451590663829706) / 0.4976497103725683
    )
    causal_links.add_causes_effects(
    ['surf_area_m2cm3', 'volume', 'P'],
    ['f6'],
    lambda surf_area_m2cm3, volume, P: 
        (((surf_area_m2cm3**2) * np.exp(-volume) * volume) * P / 
        (1 + ((surf_area_m2cm3**2) * np.exp(-volume) * volume) * P) - 
        0.04040880811128878) / 0.05992104058313283
    )
    causal_links.add_causes_effects(
    ['surf_area_m2g', 'pld', 'P'],
    ['f7'],
    lambda surf_area_m2g, pld, P: 
        ((1 / np.exp(surf_area_m2g) / pld) * P / 
        (1 + (1 / np.exp(surf_area_m2g) / pld) * P) - 
        0.033471221601783376) / 0.11285951143033723
    )
    causal_links.add_causes_effects(
    ['VolFrac', 'pld', 'P'],
    ['f8'],
    lambda VolFrac, pld, P: 
        ((VolFrac**2 / np.exp(-pld)) * P / 
        (1 + (VolFrac**2 / np.exp(-pld)) * P) - 
        0.3805783317820693) / 0.36837799819503636
    )
    causal_links.add_causes_effects(
        ['P', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8'],
        [target_name],
        lambda P, *features: sum([beta * f for (beta, f) in zip(betas, features)]) + intercept + y_train_mean - np.abs(loading_at_zero_P + y_train_mean) * epsilon_max * np.exp(-alpha * np.abs(P - 0.0)),
    )
    
    return causal_links, X_with_only_f_terms

In [ ]:
def get_causal_links_with_f_terms_only_CH4_373K():
    beta1 = 0.1909906610367201
    beta2 = 0.11682964613884445
    beta3 = -0.09344427307559947
    beta4 = 0.20074867148999848
    beta5 = 0.11122063059032497
    beta6 = 0.5214533740342101
    beta7 = -0.3427553886500203
    beta8 = -0.21426185160030492
    betas = [beta1, beta2, beta3, beta4, beta5, beta6, beta7, beta8]
    intercept = 0.0
    loading_at_zero_P = (
        -beta1*0.5657765094736966/0.19576144586981531 +
        -beta2*0.4894973861635249/0.305116996449596172 + 
        -beta3*0.054277814589412984/0.097560942436884613 + 
        -beta4*0.07338101892284041/0.088032430338732454 + 
        -beta5*0.04457244307054062/0.149505513572373745 + 
        -beta6*0.13314539459770083/0.095672598214362036 + 
        -beta7*0.11368892880920235/0.10944603062166347 + 
        -beta8*0.0539764858809039/0.173903629938781068
    )
    epsilon_max = 1.0
    y_train_mean = np.mean(y_train)
    alpha = 2.0

    causal_links = CausalLinks()
    target_name = 'y'
    X_with_only_f_terms = X[list(inspect.signature(AI_DARWIN_EQUATION).parameters.keys()) + ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8']]

    causal_links.add_causes_effects(
        ['P'],
        ['f1'],
        lambda P: 
            ((1) * P / 
            (1 + (1) * P) - 
            0.5657765094736966) / 0.1957614458698153
    )
    causal_links.add_causes_effects(
        ['largest_included_sphere_free', 'surf_area_m2cm3', 'P'],
        ['f2'],
        lambda largest_included_sphere_free, surf_area_m2cm3, P: 
            ((np.exp(-largest_included_sphere_free) * surf_area_m2cm3) * P / 
            (1 + (np.exp(-largest_included_sphere_free) * surf_area_m2cm3) * P) - 
            0.4894973861635249) / 0.30511699644959617
    )
    causal_links.add_causes_effects(
        ['NAV', 'spg', 'P'],
        ['f3'],
        lambda NAV, spg, P: 
            ((np.exp(NAV) / spg) * P / 
            (1 + (np.exp(NAV) / spg) * P) - 
            0.054277814589412984) / 0.09756094243688461
    )
    causal_links.add_causes_effects(
        ['density', 'VolFrac', 'P'],
        ['f4'],
        lambda density, VolFrac, P: 
            ((np.exp(density) * (VolFrac**2)) * P / 
            (1 + (np.exp(density) * (VolFrac**2)) * P) - 
            0.07338101892284041) / 0.08803243033873245
    )
    causal_links.add_causes_effects(
        ['largest_included_sphere_free', 'spg', 'largest_free_sphere', 'P'],
        ['f5'],
        lambda largest_included_sphere_free, spg, largest_free_sphere, P: 
            ((np.exp(largest_included_sphere_free) / (spg**2) / np.exp(largest_free_sphere)) * P / 
            (1 + (np.exp(largest_included_sphere_free) / (spg**2) / np.exp(largest_free_sphere)) * P) - 
            0.04457244307054062) / 0.14950551357237374
    )
    causal_links.add_causes_effects(
        ['void_frac', 'P'],
        ['f6'],
        lambda void_frac, P: 
            ((void_frac**2) * P / 
            (1 + (void_frac**2) * P) - 
            0.13314539459770083) / 0.09567259821436203
    )
    causal_links.add_causes_effects(
        ['VolFrac', 'void_frac', 'P'],
        ['f7'],
        lambda VolFrac, void_frac, P: 
            ((VolFrac**2 / (void_frac**2)) * P / 
            (1 + (VolFrac**2 / (void_frac**2)) * P) - 
            0.11368892880920235) / 0.1094460306216634
    )
    causal_links.add_causes_effects(
        ['surf_area_m2g', 'P'],
        ['f8'],
        lambda surf_area_m2g, P: 
            ((np.exp(-surf_area_m2g)) * P / 
            (1 + (np.exp(-surf_area_m2g)) * P) - 
            0.0539764858809039) / 0.17390362993878106
    )
    causal_links.add_causes_effects(
        ['P', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8'],
        [target_name],
        lambda P, *features: sum([beta * f for (beta, f) in zip(betas, features)]) + intercept + y_train_mean - np.abs(loading_at_zero_P + y_train_mean) * epsilon_max * np.exp(-alpha * np.abs(P - 0.0)),
    )
    
    return causal_links, X_with_only_f_terms

##### CO2

In [ ]:
def get_causal_links_with_f_terms_only_CO2_323K():
    beta1 = 0.2408158535926262
    beta2 = -0.6051468741071055
    beta3 = 0.17771546567926963
    beta4 = -2.0804716603104425
    beta5 = -0.35101203615301246
    beta6 = 1.1201332339583743
    beta7 = 0.0023376143648303396
    beta8 = -0.12134179550190091
    beta9 = -1.215940666467515
    beta10 = 2.3657034591203896
    beta11 = -0.6619785317203183
    beta12 = 1.712890108697996
    betas = [beta1, beta2, beta3, beta4, beta5, beta6, beta7, beta8, beta9, beta10, beta11, beta12]
    intercept = 0.0
    loading_at_zero_P = (
        -beta1*0.5053560148639079/0.49606356479590331 + 
        -beta2*0.022023073277147264/0.074127264455499162 + 
        -beta3*0.636492475959887/0.47883190280181783 + 
        -beta4*0.15658870051279075/0.130147357334643284 + 
        -beta5*0.4486063640488222/0.27555650106218215 + 
        -beta6*0.769527662996705/0.19142549945155396 + 
        -beta7*0.018544154536591098/0.11340313997663167 + 
        -beta8*0.9474631921424108/0.111858997802339358 + 
        -beta9*0.2259632424704171/0.128197760414795259 + 
        -beta10*0.2983059470851118/0.1531433497568306710 + 
        -beta11*0.561041654561111/0.1990428899949868411 + 
        -beta12*0.10745167163360558/0.0993918568097935512
    )
    epsilon_max = 1.0
    y_train_mean = np.mean(y_train)
    alpha = 2.0

    causal_links = CausalLinks()
    target_name = 'y'
    X_with_only_f_terms = X[list(inspect.signature(AI_DARWIN_EQUATION).parameters.keys()) + ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12']]

    causal_links.add_causes_effects(
        ['NASA', 'volume', 'P'],
        ['f1'],
        lambda NASA, volume, P: 
            ((NASA * volume) * P / 
            (1 + (NASA * volume) * P) - 
            0.5053560148639079) / 0.4960635647959033
    )

    causal_links.add_causes_effects(
        ['NAV', 'largest_free_sphere', 'largest_included_sphere', 'P'],
        ['f2'],
        lambda NAV, largest_free_sphere, largest_included_sphere, P: 
            ((NAV / np.exp(largest_free_sphere) * (largest_included_sphere**2)) * P / 
            (1 + (NAV / np.exp(largest_free_sphere) * (largest_included_sphere**2)) * P) - 
            0.022023073277147264) / 0.07412726445549916
    )

    causal_links.add_causes_effects(
        ['spg', 'NASA', 'AV', 'P'],
        ['f3'],
        lambda spg, NASA, AV, P: 
            ((np.exp(spg) * np.exp(-NASA) / np.exp(AV)) * P / 
            (1 + (np.exp(spg) * np.exp(-NASA) / np.exp(AV)) * P) - 
            0.636492475959887) / 0.4788319028018178
    )

    causal_links.add_causes_effects(
        ['density', 'VolFrac', 'P'],
        ['f4'],
        lambda density, VolFrac, P: 
            ((density * VolFrac) * P / 
            (1 + (density * VolFrac) * P) - 
            0.15658870051279075) / 0.13014735733464328
    )

    causal_links.add_causes_effects(
        ['lcd', 'largest_free_sphere', 'P'],
        ['f5'],
        lambda lcd, largest_free_sphere, P: 
            ((lcd**2 / np.exp(largest_free_sphere)) * P / 
            (1 + (lcd**2 / np.exp(largest_free_sphere)) * P) - 
            0.4486063640488222) / 0.2755565010621821
    )

    causal_links.add_causes_effects(
        ['volume', 'density', 'VolFrac', 'P'],
        ['f6'],
        lambda volume, density, VolFrac, P: 
            ((volume / np.exp(density) / np.exp(VolFrac)) * P / 
            (1 + (volume / np.exp(density) / np.exp(VolFrac)) * P) - 
            0.769527662996705) / 0.1914254994515539
    )

    causal_links.add_causes_effects(
        ['NASA', 'spg', 'P'],
        ['f7'],
        lambda NASA, spg, P: 
            ((NASA / np.exp(spg)) * P / 
            (1 + (NASA / np.exp(spg)) * P) - 
            0.018544154536591098) / 0.1134031399766316
    )

    causal_links.add_causes_effects(
        ['lcd', 'P'],
        ['f8'],
        lambda lcd, P: 
            ((lcd**2) * P / 
            (1 + (lcd**2) * P) - 
            0.9474631921424108) / 0.11185899780233935
    )

    causal_links.add_causes_effects(
        ['pld', 'volume', 'P'],
        ['f9'],
        lambda pld, volume, P: 
            ((pld / volume) * P / 
            (1 + (pld / volume) * P) - 
            0.2259632424704171) / 0.12819776041479525
    )

    causal_links.add_causes_effects(
        ['void_frac', 'P'],
        ['f10'],
        lambda void_frac, P: 
            ((void_frac) * P / 
            (1 + (void_frac) * P) - 
            0.2983059470851118) / 0.15314334975683067
    )

    causal_links.add_causes_effects(
        ['P'],
        ['f11'],
        lambda P: 
            ((1) * P / 
            (1 + (1) * P) - 
            0.561041654561111) / 0.19904288999498684
    )

    causal_links.add_causes_effects(
        ['VolFrac', 'P'],
        ['f12'],
        lambda VolFrac, P: 
            ((VolFrac) * P / 
            (1 + (VolFrac) * P) - 
            0.10745167163360558) / 0.09939185680979355
    )

    causal_links.add_causes_effects(
        ['P', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12'],
        [target_name],
        lambda P, *features: sum([beta * f for (beta, f) in zip(betas, features)]) + intercept + y_train_mean - np.abs(loading_at_zero_P + y_train_mean) * epsilon_max * np.exp(-alpha * np.abs(P - 0.0)),
    )
    
    return causal_links, X_with_only_f_terms

In [ ]:
def get_causal_links_with_f_terms_only_CO2_373K():
    beta1 = -0.679081742692596
    beta2 = 0.20307777789750459
    beta3 = 0.4329129291782464
    beta4 = 0.3464949739458949
    beta5 = 0.17098884194225522
    beta6 = 2.2905076502184967
    beta7 = -0.22028767452266362
    beta8 = -1.2269155949581563
    beta9 = -0.8108018121501624
    beta10 = 0.05328000691618266
    beta11 = -0.19349635098380905
    beta12 = 0.284054306504558
    betas = [beta1, beta2, beta3, beta4, beta5, beta6, beta7, beta8, beta9, beta10, beta11, beta12]
    intercept = 0.0
    loading_at_zero_P = (
        -beta1*0.06471130620155797/0.083122152607038291 + 
        -beta2*0.047511004998993515/0.200784779541260262 + 
        -beta3*0.011389997203239096/0.0184915493862229773 + 
        -beta4*0.5646842961313753/0.300680505660976874 + 
        -beta5*0.5136090001227659/0.195901953568235785 + 
        -beta6*0.3000617637006874/0.153364435640597656 + 
        -beta7*0.49975562767787235/0.1913659966797387 + 
        -beta8*0.24088977178365023/0.181027711464625378 + 
        -beta9*0.16738596411370837/0.106554068911640219 + 
        -beta10*0.001524646748204257/0.01437310114160613410 + 
        -beta11*0.030270422267265115/0.103962722600690711 + 
        -beta12*0.029136294307661747/0.02771544659029635512
    )
    epsilon_max = 1.0
    y_train_mean = np.mean(y_train)
    alpha = 2.0

    causal_links = CausalLinks()
    target_name = 'y'
    X_with_only_f_terms = X[list(inspect.signature(AI_DARWIN_EQUATION).parameters.keys()) + ['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12']]

    causal_links.add_causes_effects(
        ['pld', 'P'],
        ['f1'],
        lambda pld, P: 
            ((np.exp(-pld)) * P / 
            (1 + (np.exp(-pld)) * P) - 
            0.06471130620155797) / 0.08312215260703829
    )

    causal_links.add_causes_effects(
        ['surf_area_m2g', 'NASA', 'P'],
        ['f2'],
        lambda surf_area_m2g, NASA, P: 
            ((np.exp(-surf_area_m2g) * NASA) * P / 
            (1 + (np.exp(-surf_area_m2g) * NASA) * P) - 
            0.047511004998993515) / 0.20078477954126026
    )

    causal_links.add_causes_effects(
        ['largest_included_sphere', 'NAV', 'P'],
        ['f3'],
        lambda largest_included_sphere, NAV, P: 
            ((np.exp(-largest_included_sphere) * np.exp(-NAV)) * P / 
            (1 + (np.exp(-largest_included_sphere) * np.exp(-NAV)) * P) - 
            0.011389997203239096) / 0.018491549386222977
    )

    causal_links.add_causes_effects(
        ['lcd', 'volume', 'P'],
        ['f4'],
        lambda lcd, volume, P: 
            ((np.exp(-lcd) * (volume**2)) * P / 
            (1 + (np.exp(-lcd) * (volume**2)) * P) - 
            0.5646842961313753) / 0.30068050566097687
    )

    causal_links.add_causes_effects(
        ['void_frac', 'density', 'P'],
        ['f5'],
        lambda void_frac, density, P: 
            ((np.exp(void_frac) / density) * P / 
            (1 + (np.exp(void_frac) / density) * P) - 
            0.5136090001227659) / 0.19590195356823578
    )

    causal_links.add_causes_effects(
        ['NAV', 'void_frac', 'P'],
        ['f6'],
        lambda NAV, void_frac, P: 
            ((np.exp(-NAV) * void_frac) * P / 
            (1 + (np.exp(-NAV) * void_frac) * P) - 
            0.3000617637006874) / 0.15336443564059765
    )

    causal_links.add_causes_effects(
        ['void_frac', 'P'],
        ['f7'],
        lambda void_frac, P: 
            ((np.exp(-void_frac)) * P / 
            (1 + (np.exp(-void_frac)) * P) - 
            0.49975562767787235) / 0.191365996679738
    )

    causal_links.add_causes_effects(
        ['density', 'VolFrac', 'P'],
        ['f8'],
        lambda density, VolFrac, P: 
            (((density**2) * VolFrac * np.exp(VolFrac)) * P / 
            (1 + ((density**2) * VolFrac * np.exp(VolFrac)) * P) - 
            0.24088977178365023) / 0.18102771146462537
    )

    causal_links.add_causes_effects(
        ['largest_free_sphere', 'largest_included_sphere_free', 'P'],
        ['f9'],
        lambda largest_free_sphere, largest_included_sphere_free, P: 
            ((largest_free_sphere / (largest_included_sphere_free**2)) * P / 
            (1 + (largest_free_sphere / (largest_included_sphere_free**2)) * P) - 
            0.16738596411370837) / 0.10655406891164021
    )

    causal_links.add_causes_effects(
        ['NASA', 'P'],
        ['f10'],
        lambda NASA, P: 
            (((NASA**2) / np.exp(NASA)) * P / 
            (1 + ((NASA**2) / np.exp(NASA)) * P) - 
            0.001524646748204257) / 0.014373101141606134
    )

    causal_links.add_causes_effects(
        ['surf_area_m2g', 'NAV', 'P'],
        ['f11'],
        lambda surf_area_m2g, NAV, P: 
            ((surf_area_m2g * (NAV**2)) * P / 
            (1 + (surf_area_m2g * (NAV**2)) * P) - 
            0.030270422267265115) / 0.1039627226006907
    )

    causal_links.add_causes_effects(
        ['VolFrac', 'largest_free_sphere', 'P'],
        ['f12'],
        lambda VolFrac, largest_free_sphere, P: 
            ((VolFrac / largest_free_sphere) * P / 
            (1 + (VolFrac / largest_free_sphere) * P) - 
            0.029136294307661747) / 0.027715446590296355
    )

    causal_links.add_causes_effects(
        ['P', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12'],
        [target_name],
        lambda P, *features: sum([beta * f for (beta, f) in zip(betas, features)]) + intercept + y_train_mean - np.abs(loading_at_zero_P + y_train_mean) * epsilon_max * np.exp(-alpha * np.abs(P - 0.0)),
    )
    
    return causal_links, X_with_only_f_terms

##### N2

In [ ]:
def get_causal_links_with_f_terms_only_N2_323K():
    beta1 = -0.06012649624795871
    beta2 = 0.29171999178744773
    beta3 = -0.001505369599999802
    beta4 = 0.030830007929509663
    beta5 = -0.3257118942142511
    beta6 = 0.5292953747226462
    betas = [beta1, beta2, beta3, beta4, beta5, beta6]
    intercept = 0.0
    loading_at_zero_P = (
        -beta1*0.024842285545539616/0.0293150869871573431 + 
        -beta2*0.4555766667320463/0.229069661570638882 + 
        -beta3*0.7755018982379761/0.24098266572717433 + 
        -beta4*0.48063444011241646/0.47789630752610824 + 
        -beta5*0.3032790874679304/0.301721304100847745 + 
        -beta6*0.13327729792619633/0.097269266955138276  
    )
    epsilon_max = 1.0
    y_train_mean = np.mean(y_train)
    alpha = 2.0

    causal_links = CausalLinks()
    target_name = 'y'
    X_with_only_f_terms = X[list(inspect.signature(AI_DARWIN_EQUATION).parameters.keys()) + ['f1', 'f2', 'f3', 'f4', 'f5', 'f6']]

    causal_links.add_causes_effects(
        ['pld', 'void_frac', 'volume', 'P'],
        ['f1'],
        lambda pld, void_frac, volume, P: 
            ((pld**2 * void_frac / volume**2) * P / 
            (1 + (pld**2 * void_frac / volume**2) * P) - 
            0.024842285545539616) / 0.029315086987157343
    )

    causal_links.add_causes_effects(
        ['void_frac', 'lcd', 'density', 'P'],
        ['f2'],
        lambda void_frac, lcd, density, P: 
            ((void_frac * lcd / density**2) * P / 
            (1 + (void_frac * lcd / density**2) * P) - 
            0.4555766667320463) / 0.22906966157063888
    )

    causal_links.add_causes_effects(
        ['void_frac', 'spg', 'P'],
        ['f3'],
        lambda void_frac, spg, P: 
            ((void_frac**2 * spg) * P / 
            (1 + (void_frac**2 * spg) * P) - 
            0.7755018982379761) / 0.2409826657271743
    )

    causal_links.add_causes_effects(
        ['NASA', 'density', 'P'],
        ['f4'],
        lambda NASA, density, P: 
            ((NASA / density**2) * P / 
            (1 + (NASA / density**2) * P) - 
            0.48063444011241646) / 0.4778963075261082
    )

    causal_links.add_causes_effects(
        ['largest_included_sphere_free', 'lcd', 'VolFrac', 'P'],
        ['f5'],
        lambda largest_included_sphere_free, lcd, VolFrac, P: 
            ((largest_included_sphere_free * lcd * VolFrac**2) * P / 
            (1 + (largest_included_sphere_free * lcd * VolFrac**2) * P) - 
            0.3032790874679304) / 0.30172130410084774
    )

    causal_links.add_causes_effects(
        ['void_frac', 'P'],
        ['f6'],
        lambda void_frac, P: 
            ((void_frac**2) * P / 
            (1 + (void_frac**2) * P) - 
            0.13327729792619633) / 0.09726926695513827
    )

    causal_links.add_causes_effects(
        ['P', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6'],
        [target_name],
        lambda P, *features: sum([beta * f for (beta, f) in zip(betas, features)]) + intercept + y_train_mean - np.abs(loading_at_zero_P + y_train_mean) * epsilon_max * np.exp(-alpha * np.abs(P - 0.0)),
    )
    
    return causal_links, X_with_only_f_terms

In [ ]:
def get_causal_links_with_f_terms_only_N2_373K():
    beta1 = 0.07337148018041849
    beta2 = 0.47
    beta3 = -0.2230577953955047
    beta4 = 0.0414632001791709
    beta5 = 0.043098439630175324
    betas = [beta1, beta2, beta3, beta4, beta5]
    intercept = 0.0
    loading_at_zero_P = (
        -beta1*0.34510445568068854/0.158446032418603061 + 
        -beta2*0.13402139615626685/0.098079494672173422 + 
        -beta3*0.2099562104652133/0.21926657017019813 + 
        -beta4*0.43892114771426277/0.433060152769236765 + 
        -beta5*0.3362313626509776/0.3876123176749836
    )
    epsilon_max = 1.0
    y_train_mean = np.mean(y_train)
    alpha = 2.0

    causal_links = CausalLinks()
    target_name = 'y'
    X_with_only_f_terms = X[list(inspect.signature(AI_DARWIN_EQUATION).parameters.keys()) + ['f1', 'f2', 'f3', 'f4', 'f5']]

    causal_links.add_causes_effects(
        ['density', 'P'],
        ['f1'],
        lambda density, P: 
            ((1 / density**2) * P / 
            (1 + (1 / density**2) * P) - 
            0.34510445568068854) / 0.15844603241860306
    )

    causal_links.add_causes_effects(
        ['void_frac', 'P'],
        ['f2'],
        lambda void_frac, P: 
            ((void_frac**2) * P / 
            (1 + (void_frac**2) * P) - 
            0.13402139615626685) / 0.09807949467217342
    )

    causal_links.add_causes_effects(
        ['VolFrac', 'volume', 'P'],
        ['f3'],
        lambda VolFrac, volume, P: 
            ((VolFrac**2 * volume) * P / 
            (1 + (VolFrac**2 * volume) * P) - 
            0.2099562104652133) / 0.2192665701701981
    )

    causal_links.add_causes_effects(
        ['ASA', 'VolFrac', 'P'],
        ['f4'],
        lambda ASA, VolFrac, P: 
            ((ASA * VolFrac**2) * P / 
            (1 + (ASA * VolFrac**2) * P) - 
            0.43892114771426277) / 0.43306015276923676
    )

    causal_links.add_causes_effects(
        ['NASA', 'largest_included_sphere', 'volume', 'P'],
        ['f5'],
        lambda NASA, largest_included_sphere, volume, P: 
            ((NASA * largest_included_sphere / volume**2) * P / 
            (1 + (NASA * largest_included_sphere / volume**2) * P) - 
            0.3362313626509776) / 0.387612317674983
    )

    causal_links.add_causes_effects(
        ['P', 'f1', 'f2', 'f3', 'f4', 'f5'],
        [target_name],
        lambda P, *features: sum([beta * f for (beta, f) in zip(betas, features)]) + intercept + y_train_mean + np.abs(loading_at_zero_P + y_train_mean) * epsilon_max * np.exp(-alpha * np.abs(P - 0.0)),
    )
    
    return causal_links, X_with_only_f_terms

##### Function Map

In [ ]:
function_map_causal_links_with_f_terms_only = {
    'CH4': {
        323: get_causal_links_with_f_terms_only_CH4_323K,
        373: get_causal_links_with_f_terms_only_CH4_373K,
    },
    'CO2': {
        323: get_causal_links_with_f_terms_only_CO2_323K,
        373: get_causal_links_with_f_terms_only_CO2_373K,
    },
    'N2': {
        323: get_causal_links_with_f_terms_only_N2_323K,
        373: get_causal_links_with_f_terms_only_N2_373K,
    }
}
GET_CAUSAL_LINKS_WITH_F_TERMS_ONLY = function_map_causal_links_with_f_terms_only[ADSORBATE][TEMPERATURE]

#### Generate Simplified Shapley Flow Graph

In [ ]:
causal_links, X_with_only_f_terms = GET_CAUSAL_LINKS_WITH_F_TERMS_ONLY()
causal_graph_simplified = build_feature_graph(X_with_only_f_terms, causal_links, target_name=target_name)
causal_graph_simplified.draw(rankdir='TB')

nruns = 100 # number of monte carlo samples
sample_ind = 0
bg = ADSORBATE_DATA[BACKGROUND_ADSORBENT][TEMPERATURE].iloc[[PRESSURE_IDX]].drop(labels=['y'], axis=1)
fg = X.iloc[[PRESSURE_IDX]] # foreground samples to explain. Selecting row from X such that the pressure value is equal to the pressure value of bg
bg = bg[fg.columns] # ensure fg and bg have same column order. NOTE: this only works when there is 1 fg

cf_flow = GraphExplainer(causal_graph_simplified, bg, nruns=nruns).shap_values(fg, rankdir='TB')
cf_flow.fold_noise = True
shapflow_graph_simplified = cf_flow.draw(sample_ind, show_fg_val=False)#, rankdir='TB')
# shapflow_graph_simplified = run_shapflow_without_displaying_graph(causal_graph_simplified, bg, fg, nruns=nruns, sample_ind=sample_ind)

### Shapley Flow For Every Permutation of Zeolites

In [ ]:
def get_foreground_and_background_samples(x, background_adsorbent, temperature, pressure_idx):
    bg = ADSORBATE_DATA[background_adsorbent][temperature].iloc[[pressure_idx]].drop(labels=['y'], axis=1)
    fg = x.iloc[[pressure_idx]] # foreground samples to explain. Selecting row from X such that the pressure value is equal to the pressure value of bg
    bg = bg[fg.columns] # ensure fg and bg have same column order. NOTE: this only works when there is 1 fg
    return fg, bg

def run_shapflow(causal_graph, bg, fg, nruns=100, sample_ind=0, rankdir='LR'):
    shapflow = GraphExplainer(causal_graph, bg, nruns=nruns).shap_values(fg, rankdir=rankdir)
    shapflow.fold_noise = True
    shapflow_graph = shapflow.draw(sample_ind, show_fg_val=False)
    return shapflow_graph

def run_shapflow_without_displaying_graph(causal_graph, bg, fg, nruns=100, sample_ind=0, rankdir='LR'):
    shapflow = GraphExplainer(causal_graph, bg, nruns=nruns).shap_values(fg, rankdir=rankdir)
    shapflow.fold_noise = True
    shapflow_graph = cf_flow.credit2dot(
            cf_flow.edge_credit,
            format_str="{:.2f}",
            idx=sample_ind,
            max_display=None,
            show_fg_val=False
        )
    return shapflow_graph

def save_shapflow_graph(save_dir, graph_name, graph, foreground_adsorbent, background_adsorbent):
    os.makedirs(save_dir, exist_ok=True)
    graph.draw(
        path=os.path.join(save_dir, f'{graph_name}_{background_adsorbent}_to_{foreground_adsorbent}.svg'),
        format='svg',
        prog='dot'
    )


In [ ]:
i = 0
nruns = 100 # number of monte carlo samples
sample_ind = 0
dir_name = 'zeolite'
save_dir = f'results/{dir_name}/{ADSORBATE}/{TEMPERATURE}'
failed_pairs = []
for foreground_adsorbent in common_zeolites:
    for background_adsorbent in common_zeolites:
        if foreground_adsorbent == background_adsorbent:
            continue
        
        print(f'{foreground_adsorbent=} vs {background_adsorbent=}')
        try:
            _X = ADSORBATE_DATA[foreground_adsorbent][TEMPERATURE]
            X = _X.drop(labels=['y'], axis=1)
            causal_graph = build_causal_graph(y, X, target_name, debug=False)
            fg, bg = get_foreground_and_background_samples(X, background_adsorbent, TEMPERATURE, PRESSURE_IDX)
            shapflow_graph = run_shapflow_without_displaying_graph(causal_graph, bg, fg, nruns=nruns, sample_ind=sample_ind, rankdir='LR')
            save_shapflow_graph(save_dir, 'shapflow_graph', shapflow_graph, foreground_adsorbent, background_adsorbent)

            causal_links, X_with_only_f_terms = GET_CAUSAL_LINKS_WITH_F_TERMS_ONLY()
            causal_graph_simplified = build_feature_graph(X_with_only_f_terms, causal_links, target_name=target_name)
            fg, bg = get_foreground_and_background_samples(X_with_only_f_terms, background_adsorbent, TEMPERATURE, PRESSURE_IDX)
            shapflow_graph_simplified = run_shapflow_without_displaying_graph(causal_graph_simplified, bg, fg, nruns=nruns, sample_ind=sample_ind, rankdir='TB')
            save_shapflow_graph(save_dir, 'shapflow_graph_simplified', shapflow_graph_simplified, foreground_adsorbent, background_adsorbent)
        except Exception as e:
            failed_pairs.append((foreground_adsorbent, background_adsorbent, str(e)))
            print(f'Error: {e}')

In [ ]:
failed_pairs

## Save Results

In [ ]:
dir_name = 'zeolite'
save_dir = f'results/{dir_name}/{ADSORBATE}/{TEMPERATURE}'
os.makedirs(save_dir, exist_ok=True)

computational_graph.render(os.path.join(save_dir, 'computational_graph'), format='svg')
causal_graph.to_graphviz(rankdir='LR').draw(
    path=os.path.join(save_dir, 'causal_graph.svg'),
    format='svg',
    prog='dot'
)
shapflow_graph_with_constant_pressure.draw(
    path=os.path.join(save_dir, f'shapflow_graph_{BACKGROUND_ADSORBENT}_to_{FOREGROUND_ADSORBENT}.svg'),
    format='svg',
    prog='dot'
)
causal_graph_simplified.to_graphviz(rankdir='LR').draw(
    path=os.path.join(save_dir, 'causal_graph_simplified.svg'),
    format='svg',
    prog='dot'
)
shapflow_graph_simplified.draw(
    path=os.path.join(save_dir, f'shapflow_graph_simplified_{BACKGROUND_ADSORBENT}_to_{FOREGROUND_ADSORBENT}.svg'),
    format='svg',
    prog='dot'
)
print("All graphs have been saved to:", save_dir)

In [ ]:
shapflow_graph_simplified.draw(
    path=f'shapflow_graph_simplified_{BACKGROUND_ADSORBENT}_to_{FOREGROUND_ADSORBENT}.svg',
    format='svg',
    prog='dot'
)